In [14]:
from avian_vocalizations import data
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from avian_vocalizations.data import AudioFeatureGenerator
from sklearn.preprocessing import LabelEncoder
import os
from glob import glob
import numpy as np

label_encoder = LabelEncoder().fit(index_df['english_cname'] )

data_dir = '../data'
index_df, shapes_df, train_df, test_df = data.load_data(data_dir)

n_classes = len(label_encoder.classes_)
print("The dataset contains %i distinct species labels."%n_classes)
mp3_dir = os.path.join(data_dir,'audio')
print("%i mp3s found in %s/"%(len(glob(os.path.join(mp3_dir,'*.mp3'))), os.path.abspath(mp3_dir)))


X_train = index_df.loc[index_df['test']==False].index.values
y_train = label_encoder.transform(index_df.loc[index_df['test']==False,"english_cname"].values)

The dataset contains 91 distinct species labels.
2730 mp3s found in /home/sam/Downloads/avian_vocalizations/data/audio/


In [15]:
training_generator = AudioFeatureGenerator(X_train, y_train, batch_size=len(X_train), 
                                           shuffle=True, seed=37, n_frames=128, data_dir=data_dir,
                                           n_classes=n_classes)
scores=[]
nb = GaussianNB()
Xs, ys = training_generator[0] #  batch_size=len(X_test), so just the first batch
Xs = Xs.reshape(Xs.shape[0],Xs.shape[1]*Xs.shape[2])
ys = np.argmax(ys,axis=1)
nb.partial_fit(Xs, ys, classes=range(n_classes))
predictions = nb.predict(Xs) 
training_accuracy = accuracy_score(ys, predictions)
print("Training accuracy of benchmark model: %.5f"%training_accuracy)

Training accuracy of benchmark model: 0.18297


In [18]:
X_test = index_df.loc[index_df['test']==True].index.values
y_test = label_encoder.transform(index_df.loc[index_df['test']==True,"english_cname"].values)

test_generator = AudioFeatureGenerator(X_test, y_test, batch_size=len(X_test), data_dir=data_dir,
                                       seed=37, n_frames=128, n_classes=n_classes)
Xs, ys = test_generator[0] # batch_size=len(X_test), so just the first batch
Xs = Xs.reshape(Xs.shape[0],Xs.shape[1]*Xs.shape[2])
ys = np.argmax(ys,axis=1)
predictions = nb.predict(Xs) 
test_accuracy = accuracy_score(ys, predictions)
print("Test accuracy of benchmark model: %.5f"%test_accuracy)

Test accuracy of benchmark model: 0.05934
